In [7]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn import metrics
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

PATH = '/Users/krc/Desktop/project/system_quality_project/data/'

In [9]:
train_err = pd.read_csv(PATH + 'train_err_data.csv')
id_error = train_err[['user_id','errtype']].values
error = np.zeros((15000,42))
for person_idx, err in tqdm(id_error):
    error[person_idx - 10000,err - 1] += 1

train_prob = pd.read_csv(PATH + 'train_problem_data.csv')
problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1

train = pd.DataFrame(data=error)
train['problem'] = problem
del error,problem

clf = setup(data = train, target='problem')

100%|██████████████████████████| 16554663/16554663 [00:14<00:00, 1130080.16it/s]


,Description,Value
0,Session id,6180
1,Target,problem
2,Target type,Binary
3,Original data shape,"(15000, 43)"
4,Transformed data shape,"(15000, 43)"
5,Transformed train set shape,"(10499, 43)"
6,Transformed test set shape,"(4501, 43)"
7,Numeric features,42
8,Preprocess,True
9,Imputation type,simple


In [10]:
best_5 = compare_models(sort='Accuracy' , n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7786,0.8006,0.4951,0.7564,0.5983,0.4546,0.4741,0.1160
gbc,Gradient Boosting Classifier,0.7761,0.8007,0.4583,0.7789,0.5767,0.4382,0.4668,0.4290
et,Extra Trees Classifier,0.7756,0.7977,0.4806,0.7579,0.5880,0.4443,0.4661,0.1850
rf,Random Forest Classifier,0.7728,0.7988,0.4686,0.7574,0.5788,0.4349,0.4585,0.2190
ada,Ada Boost Classifier,0.7710,0.7901,0.4634,0.7551,0.5742,0.4297,0.4537,0.1150
lr,Logistic Regression,0.7507,0.7330,0.3646,0.7639,0.4934,0.3547,0.3974,0.3430
qda,Quadratic Discriminant Analysis,0.7502,0.7349,0.3794,0.7467,0.5029,0.3590,0.3958,0.0270
lda,Linear Discriminant Analysis,0.7500,0.7464,0.3329,0.8010,0.4701,0.3412,0.3977,0.0300
ridge,Ridge Classifier,0.7494,0.0000,0.3254,0.8087,0.4639,0.3372,0.3968,0.0770
nb,Naive Bayes,0.7332,0.7402,0.2800,0.7752,0.4102,0.2850,0.3467,0.0230


In [11]:
blended = blend_models(estimator_list= best_5 , fold= 5, method='soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7824,0.8145,0.4757,0.7872,0.5931,0.4566,0.4836
1,0.7681,0.7897,0.4557,0.7506,0.5671,0.4214,0.4459
2,0.7743,0.7956,0.4471,0.7825,0.5691,0.4312,0.4622
3,0.7829,0.8132,0.4900,0.7760,0.6007,0.4618,0.4849
4,0.7956,0.8175,0.4957,0.8203,0.6180,0.4898,0.5188
Mean,0.7806,0.8061,0.4729,0.7833,0.5896,0.4522,0.4791
Std,0.0093,0.0112,0.0189,0.0224,0.0193,0.0242,0.0246


In [12]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7805,0.8066,0.4527,0.8026,0.5789,0.4456,0.4790


In [13]:
final_model = finalize_model(blended)

In [23]:
final_model

VotingClassifier(estimators=[('Light Gradient Boosting Machine',
                              LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=6180, reg_alpha=0.0,
                                             reg_lamb...
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                                     n_jobs=-1, oob_score=False,
                                                     random_state=6180,
                                                     verbose=0,
                                                     warm_start=False)),
                             ('Ada Boost Classifier',
                              AdaBoostClassifier(algorithm='SAMME.R',
                                                 base_estimator=None,
                                                 learning_rate=1.0,
                                                 n_estimators=50,
                                                 random_state=6180))],
                 flatten_transform=True, n_jobs=-1, verbose=False,
                 voting='soft', weights=None)

In [14]:
test_err = pd.read_csv(PATH + 'test_err_data.csv')
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((14999,42))
for person_idx , err in tqdm(id_error):
    test_x[person_idx - 30000, err -1] += 1


100%|██████████████████████████| 16532648/16532648 [00:14<00:00, 1103334.59it/s]


(14999, 42)

In [20]:
test = pd.DataFrame(data=test_x)

In [21]:
predictions = predict_model(final_model,data = test)

In [26]:
predictions.Label[0]

1

In [28]:
x= []

for i in range(len(predictions['Score'])):
    if predictions['Label'][i] == 1:
        x.append(predictions['Score'][i])
    else:
        x.append(1-predictions['Score'][i])
x

[0.821,
 0.30589999999999995,
 0.28590000000000004,
 0.6915,
 0.5881,
 0.43889999999999996,
 0.20189999999999997,
 0.13029999999999997,
 0.6097,
 0.3274,
 0.2015,
 0.23050000000000004,
 0.8261,
 0.21140000000000003,
 0.20130000000000003,
 0.3345,
 0.3308,
 0.4677,
 0.384,
 0.36339999999999995,
 0.7427,
 0.5223,
 0.2672,
 0.17889999999999995,
 0.5082,
 0.37470000000000003,
 0.38639999999999997,
 0.2147,
 0.28049999999999997,
 0.16110000000000002,
 0.22929999999999995,
 0.3306,
 0.2208,
 0.24739999999999995,
 0.41890000000000005,
 0.3125,
 0.18489999999999995,
 0.4012,
 0.2845,
 0.16620000000000001,
 0.39880000000000004,
 0.23340000000000005,
 0.15590000000000004,
 0.14649999999999996,
 0.20389999999999997,
 0.3772,
 0.3993,
 0.2319,
 0.3163,
 0.17659999999999998,
 0.23850000000000005,
 0.28969999999999996,
 0.3477,
 0.18689999999999996,
 0.12929999999999997,
 0.27180000000000004,
 0.35540000000000005,
 0.37829999999999997,
 0.4275,
 0.26480000000000004,
 0.5046,
 0.7995,
 0.8299,
 0.126

In [30]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv(PATH+"AutoML.csv", index = False)